# UR Arm ROS2 Control

## Joint Control

First you need to save your Joints Module.  The UR tells its current joint states on the topic "joint_states" and lets you set the angles with the action "/follow_joint_trajectory".

In [ ]:
%%writefile Joints.py

import rclpy
from rclpy.node import Node
from rclpy.action import ActionClient
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from control_msgs.action import FollowJointTrajectory
import math

class Joints(rclpy.node.Node):
    """
    Subscribe to the joint_states topic.
    """

    def __init__(self):
        super().__init__("joint_state_subscriber")
        self.subscription = self.create_subscription(JointState, "joint_states", self.joints_callback, 10)
        self._action_client = ActionClient(self, FollowJointTrajectory, "scaled_joint_trajectory_controller/follow_joint_trajectory")
        if not self._action_client.wait_for_server(timeout_sec=10):
            rclpy.shutdown()
            raise RuntimeError(
                "Action server not available after waiting for 10 seconds. Check ROS UR Driver."
            )
        self.states = None
        self.done = False

    def joints_callback(self, msg):
        self.states = {"name": msg.name, "position": msg.position, "velocity": msg.velocity, "effort": msg.effort}
        #    "name": [msg.name[5]] + msg.name[0:5],
        #    "position": [msg.position[5]] + msg.position[0:5].tolist(),
        #    "velocity": [msg.velocity[5]] + msg.velocity[0:5].tolist(),
        #    "effort": [msg.effort[5]] + msg.effort[0:5].tolist(),}

    def read(self):
        return self.states

    def move_once(self, joint_positions, time = 5, degrees = True):
        trajectory = JointTrajectory()
        trajectory.joint_names = ["shoulder_pan_joint","shoulder_lift_joint","elbow_joint","wrist_1_joint","wrist_2_joint","wrist_3_joint"]
        trajectory.points.append(set_waypoint(joint_positions, time, degrees))
        self.move_arm(trajectory)

    def move_many(self, q_matrix, time, degrees = True):
        trajectory = JointTrajectory()
        trajectory.joint_names = ["shoulder_pan_joint","shoulder_lift_joint","elbow_joint","wrist_1_joint","wrist_2_joint","wrist_3_joint"]
        for q in q_matrix:
            next_time =  time + i * (time / (len(q_matrix) - 1))
            trajectory.points.append(set_waypoint(q, next_time, degrees))
        self.move_arm(trajectory)

    def set_waypoint(self, joint_positions, time = 5, degrees = True)::
        point = JointTrajectoryPoint
        if degrees: 
            joint_positions = [math.to_radians(q) for q in joint_positions]
        point.positions =  joint_positions
        point.time_from_start = Duration(sec = int(time), nanosec = int(time % 1 * 1000000000))
        return point
        
    def move_arm(self, trajectory):
        goal_msg = FollowJointTrajectory.Goal()
        goal_msg.trajectory = trajectory

        self._action_client.wait_for_server()
        self._set_goal = self._action_client.send_goal_async(goal_msg)
        self._set_goal.add_done_callback(self.goal_request_callback)
        self.get_logger().debug(f"Sent trajectory {trajectory}")
        
    def goal_request_callback(self, future): 
        '''
        run this when the action server responds with either go or no-go and set up another callback for when the action is done
        '''
        goal_handle = future.result()
        if not goal_handle.accepted:
            self.get_logger().info('Goal rejected :(')
            self.done = True
            return
        self.get_logger().info('Goal accepted :)')
        # goal accepted - now wait for it to be executed
        self._when_done = goal_handle.get_result_async()
        self._when_done.add_done_callback(self.get_result_callback)

    def get_result_callback(self, future):
        '''
        run this when the action is done
        '''        
        result = future.result().result
        self.get_logger().info('Result: {0}'.format(result))
        self.done = True
        
    def stop(self):
        positions = self.read()
        point = JointTrajectoryPoint()
        point.positions = positions
        point.time_from_start = Duration(sec=0, nanosec=100000000)
        trajectory.points.append(point)
        self.move_arm(trajectory)
               


And a sample code

In [ ]:
import rclpy
from Joints import Joints
import time

rclpy.init()
j = Joints()
print(j.read())
j.move_once([0,0,0,0,0,0])
while not j.done:
    time.sleep(1)
rclpy.shutdown()

In [5]:
f = [[0,2,2],[1,3,5]]
print(len(f))


2


In [3]:
print(int(5.6))

5
